# Основной пайплайн Video Dubbing AI

1. Пути к данным
2. Загрузка сегментов (ASR)
3. Перевод (MT)
4. Генерация аудио (TTS)
5. Сборка итоговой аудиодорожки
6. Замена аудиодорожки в видео
7. Визуализация

## 1. Пути к данным

In [ ]:
import sys, os
print('Python executable:', sys.executable)
print('Если ядро не совпадает с venv — выберите вручную!')

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)
print('src_path:', src_path)
print('sys.path:', sys.path)

## 2. Загрузка сегментов (ASR)

In [ ]:
import glob, re, os

output_dir = '../data/output'
pattern = os.path.join(output_dir, 'asr_segments_ru_*.txt')
files = glob.glob(pattern)
valid_files = [f for f in files if re.match(r'asr_segments_ru_\d{8}_\d{4}\.txt$', os.path.basename(f)) and os.path.getsize(f) > 1024]
if not valid_files:
    raise FileNotFoundError('Не найден ни один валидный файл перевода сегментов в ../data/output/')
asr_path = sorted(valid_files, key=os.path.getmtime)[-1]
with open(asr_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

segments = []
for line in lines:
    parts = line.strip().split('\t')
    if len(parts) < 3:
        continue
    start, end, text = parts[0], parts[1], '\t'.join(parts[2:])
    try:
        segments.append({
            'start': float(start.replace(',', '.')),
            'end': float(end.replace(',', '.')),
            'text': text
        })
    except Exception as e:
        print(f"Ошибка парсинга строки: {line} — {e}")

print(f'Загружено сегментов: {len(segments)} из {asr_path}')

## 3. Перевод (MT)

In [ ]:
from mt.translator import Translator
mt = Translator()
ru_segments = []
for seg in segments:
    ru_text = mt.translate(seg['text'], src_lang='en', tgt_lang='ru', use_gpu=False)
    ru_segments.append({'start': seg['start'], 'end': seg['end'], 'text': ru_text})
print(f'Переведено сегментов: {len(ru_segments)}')

## 4. Генерация аудио (TTS)

In [ ]:
from tts.voice_cloner import VoiceCloner
tts = VoiceCloner('../data/input/reference_audio.wav')
audio_paths = []
for i, seg in enumerate(ru_segments):
    out_path = f'../data/temp/run_tts_segment_{i}.wav'
    tts.synthesize(seg['text'], output_path=out_path, use_gpu=False)
    audio_paths.append(out_path)
print(f'Сгенерировано аудиосегментов: {len(audio_paths)}')

## 5. Сборка итоговой аудиодорожки

In [ ]:
from vision_audio.video_processor import assemble_audio_by_segments
processed_segments = [
    {'start': seg['start'], 'end': seg['end'], 'audio_path': path}
    for seg, path in zip(ru_segments, audio_paths)
]
final_audio_path = '../data/temp/run_assembled_audio.wav'
assemble_audio_by_segments(processed_segments, base_audio_path='../data/input/extracted_audio.wav', output_path=final_audio_path)
print(f'📁 Итоговое аудио: {final_audio_path}')

## 6. Замена аудиодорожки в видео

In [ ]:
from vision_audio.video_processor import VideoProcessor
input_video = '../data/input/input.mp4'
output_video = '../data/output/output_zero_shot_latest.mp4'
VideoProcessor.replace_audio(input_video, final_audio_path, output_video)
print(f'Готово! Видео сохранено: {output_video}')

## 7. Визуализация
### График длительности сегментов

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
if not ru_segments:
    print('❌ Ошибка: ru_segments пустой список!')
else:
    durations = [seg['end'] - seg['start'] for seg in ru_segments]
    plt.figure(figsize=(12, 6))
    plt.bar(range(len(durations)), durations, alpha=0.7, color='skyblue')
    plt.xlabel('Номер сегмента')
    plt.ylabel('Длительность (секунды)')
    plt.title(f'Длительность сегментов (всего: {len(durations)})')
    plt.grid(True, alpha=0.3)
    avg_duration = sum(durations) / len(durations)
    plt.axhline(y=avg_duration, color='red', linestyle='--', label=f'Средняя длительность: {avg_duration:.2f}s')
    plt.legend()
    plt.tight_layout()
    plt.savefig('segment_durations.png')
    plt.show()
    print('График сохранён в файл segment_durations.png')

### Аудиоплеер

In [ ]:
from IPython.display import Audio
Audio(final_audio_path)

### Видеоплеер

In [ ]:
from IPython.display import Video
import glob, os
output_dir = '../data/output'
mp4_files = glob.glob(os.path.join(output_dir, 'output*.mp4'))
if not mp4_files:
    raise FileNotFoundError('Не найдено ни одного итогового видео в ../data/output/')
mp4_files.sort(key=os.path.getmtime, reverse=True)
last_video = mp4_files[0]
Video(last_video, embed=True)